### Sequential Chain

##### Boilerplate code

In [2]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)


In [8]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# First chain
first_prompt = ChatPromptTemplate.from_template(
    "How much is {a} + {b}. Answer only with the result\nResult: "
)
first_chain = first_prompt | google_llm

# Second chain
second_prompt = ChatPromptTemplate.from_template(
    "How much is {a} times {c}"
)
second_chain = second_prompt | google_llm

# Complete sequential chain using LCEL
# complete_chain = (
#     {   
#         "a": itemgetter("a"), # Pass 'a' to next chain
#         "c": first_chain  # Save result in variable 'c'
           
#     }
#     |
#     RunnablePassthrough.assign(d=second_chain)
#     | second_chain  # Run second chain with 'a' and 'c'
# )

complete_chain = (
    {
        "a": itemgetter("a"),
        "b": itemgetter("b"),
        "c": first_chain
    }
    | second_chain
)

result = complete_chain.invoke({'a': 2, 'b': 3})
print(result)


content='2 times 5 is 10.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--b65155ca-ea24-4407-b3c8-82671c6a33f8-0' usage_metadata={'input_tokens': 150, 'output_tokens': 10, 'total_tokens': 160, 'input_token_details': {'cache_read': 0}}
